In [1]:
import os.path
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances

import re #regular expression matching for removing unwanted columns by name 
import natsort as ns #3rd party package for natural sorting 

In [2]:
def raw_data_cleanup(filename):
    
    """
    Imports RNAseq .csv file and does basic clean up of "FM40" 
        -sorts FM40 timecourse sequence chronologically
        -removes all QC data and non FM40 columns
        -returns dataframe with locus tag set as index
    """

    if os.path.isfile(filename):
        print("{} was located in the directory".format(filename))
                
#import the data
        data0_raw = pd.read_csv(filename, sep = "\t") 
        print("{} was imported into dataframe".format(filename))
        
#removing all QC data
        data1_noQC = data0_raw.select(lambda x: not re.search("QC", x), axis = 1) 
        print("QC columns were removed from dataframe")

#removing all non FM40 data
        data2_FM40only = data1_noQC.select(lambda x: re.search("FM40", x), axis = 1)
        print("All non FM40 data were removed from dataframe")
        
#naturally sorting FM40 data by columns 
        cols = list(ns.natsorted(data2_FM40only.columns))
        data3_sorted=data2_FM40only[cols]
        print("All FM40 columns were sorted by timecourse sequence")
        
#adding the descriptor columns back to FM40
        qualitative = data0_raw.loc[:,"locus_tag":"translation"]
        data4_sorted = pd.concat([qualitative, data3_sorted], axis = 1)

#setting locus tag to be the index
        data5_index = data4_sorted.set_index("locus_tag")
        
    
        print("Clean-up of raw data complete")
        return data5_index
        
    else:
        print("{} does not exist in directory. Function was not complete.".format(filename))
        return
    

In [3]:
def TPM_counts(dataframe, 
              gene_start,
              gene_stop,
              columns):
    
    """
    TPM_counts(dataframe, gene_start, gene_stop, columns):

    returns a dataframe with TPM instead of reads

    Parameters
    ----------
    daraframe = dataframe object variable
    gene_start = string with column name containing gene start coordinate
    gene_stop = string with column name containing gene stop coordinate
    columns = list of strings of column names to be converted to TPM


    Run the following two lines to properly execute this function:

    columns = ['5GB1_FM40_T0m_TR2', '5GB1_FM40_T10m_TR3', '5GB1_FM40_T20m_TR2', '5GB1_FM40_T40m_TR1',
           '5GB1_FM40_T60m_TR1', '5GB1_FM40_T90m_TR2', '5GB1_FM40_T150m_TR1_remake', '5GB1_FM40_T180m_TR1']

    TPM_counts(df,"start_coord","end_coord",columns)
    """
    
    #create empty dataframe 
    gene_length = pd.DataFrame()
    
    #gene length in kilo base pairs as new column
    gene_length["gene_length"] = (dataframe[gene_stop]- dataframe[gene_start] + 1)/1000   
    
    #normalize read counts by gene length in kilo base pairs
    RPK = dataframe.loc[:,columns].div(gene_length.gene_length, axis=0) 
    
    #creating a series with the sums of each FM40 column / 1,000,000
    norm_sum = RPK.sum(axis=0)/1000000 
    norm_sum1 = pd.Series.to_frame(norm_sum)
    norm_sum2 = norm_sum1.T
    
    #dividing by the the total transcript counts in each repicate
    TPM = RPK.div(norm_sum2.ix[0]) 
    
    dataframe.loc[:,columns] = TPM
    
    return dataframe 



### Log2 fold transfrom of the TPM data.

In [4]:
def log_2_transform(dataframe,
                    first_data_column,
                    last_data_column):
                  
    """
    log_2_transform(dataframe, 
                    first_data_column, 
                    last_data_column)
    
    Return a new dataframe with the range of data columns log2 transformed. 
    *all zero values are changed to 1 (yield 0 after transform)
    *all values less than 1 are changed to 1 (yield 0 after transform)
    
    Parameters
    ----------
    daraframe = dataframe object variable
    first_data_column = first column that contains actual data (first non categorical)
    last_data_column = last column taht contains actual data (last non categorigal column)

    Run the following to execute the function for Cu transition dataset. 

    log_2_transform(df, "5GB1_FM40_T0m_TR2", "5GB1_FM40_T180m_TR1") 
    
    """
    
    df_data = dataframe.loc[:,first_data_column:last_data_column] #isolate the data
    
    df_data = df_data.replace(0,1) #replace all zeros with 1s
    
    df_data[df_data<1] = 1 #replace all values less than 1 with 1
    
    df_data_log2 = df_data.apply(np.log2)
    
    return df_data_log2

### Mean center the data 

In [5]:
def mean_center(df, first_data_column, last_data_column):       
        
        
    """
    mean_center(dataframe, 
                first_data_column, 
                last_data_column)
    
    Return a new dataframe with the range of data columns log2 transformed. 
    
    Parameters
    ----------
    daraframe = dataframe object variable
    first_data_column = first column that contains actual data (first non categorical)
    last_data_column = last column taht contains actual data (last non categorigal column)

    Run the following to execute the function for Cu transition dataset. 

    mean_center(df, "5GB1_FM40_T0m_TR2", "5GB1_FM40_T180m_TR1") 
    
    """

    df2_TPM_values = df2_TPM.loc[:,first_data_column:last_data_column] #isolating the data values 
    df2_TPM_values_T = df2_TPM_values.T #transposing the data

    standard_scaler = StandardScaler(with_std=False)
    TPM_counts_mean_centered = standard_scaler.fit_transform(df2_TPM_values_T) #mean centering the data 

    TPM_counts_mean_centered = pd.DataFrame(TPM_counts_mean_centered) #back to Dataframe

    #transposing back to original form and reincerting indeces and columns 
    my_index = df2_TPM_values.index
    my_columns = df2_TPM_values.columns

    TPM_counts_mean_centered = TPM_counts_mean_centered.T
    TPM_counts_mean_centered.set_index(my_index, inplace=True)
    TPM_counts_mean_centered.columns = my_columns
    
    return TPM_counts_mean_centered

### Pariwise distance metric table - euclidean distance 

In [6]:
def euclidean_distance(dataframe, first_data_column, last_data_column):
    
    """
    euclidean_distance(dataframe, 
                first_data_column, 
                last_data_column)
    
    Return a new dataframe - pairwise distance metric table, euclidean distance between every pair of rows. 
    
    Parameters
    ----------
    daraframe = dataframe object variable
    first_data_column = first column that contains actual data (first non categorical)
    last_data_column = last column taht contains actual data (last non categorigal column)

    Run the following to execute the function for Cu transition dataset. 

    euclidean_distance(df, "5GB1_FM40_T0m_TR2", "5GB1_FM40_T180m_TR1") 
    
    """
    
    
    df_values = dataframe.loc[:,first_data_column:last_data_column] #isolating the data values 

    df_euclidean_distance = pd.DataFrame(euclidean_distances(df_values))

    my_index = dataframe.index
    
    df_euclidean_distance = df_euclidean_distance.set_index(my_index)
    df_euclidean_distance.columns = my_index
    
    return df_euclidean_distance

In [7]:
def congruency_table(df, 
         data_clm_strt, 
         data_clm_stop, 
         step, 
         mask_diagonal=False):
    
    """
    
    corr(df, data_clm_strt, data_clm_stop, step = len(df.columns), mask_diagonal=False)
    
    returns a new datafram - congruency table - a pairwise pearson correlation matrix for every row pair
    
    Parameters
    ----------
    
    df - dataframe argument - recommended to use TPM counts for RNAseq datasets. 
    data_clm_strt = first column that contains data to be processed 
    data_clm_stop = last column that contains data to be processed
    step = length of dataset 
    mask_diagonal = mask diagonal values which shoud come out as 1
    
    
    Run the following lines to execute the function for my data
    
    congruency_table(df1, "5GB1_FM40_T0m_TR2" , "5GB1_FM40_T180m_TR1")
    
    """
    
    df = df.loc[:, data_clm_strt: data_clm_stop] #isolating the rows that are relavent to us. 
    df = df.T 
    
    n = df.shape[0]

    def corr_closure(df):
        d = df.values
        sums = d.sum(0, keepdims=True)
        stds = d.std(0, keepdims=True)

        def corr_(k=0, l=10):
            d2 = d.T.dot(d[:, k:l])
            sums2 = sums.T.dot(sums[:, k:l])
            stds2 = stds.T.dot(stds[:, k:l])

            return pd.DataFrame((d2 - sums2 / n) / stds2 / n,
                                df.columns, df.columns[k:l])

        return corr_

    c = corr_closure(df)

    step = min(step, df.shape[1])

    tups = zip(range(0, n, step), range(step, n + step, step))

    corr_table = pd.concat([c(*t) for t in tups], axis=1)

    if mask_diagonal:
        np.fill_diagonal(corr_table.values, np.nan)

    return corr_table

In [8]:
df1_raw_FM40 = raw_data_cleanup("5G_counts.tsv")


columns = ['5GB1_FM40_T0m_TR2', '5GB1_FM40_T10m_TR3', '5GB1_FM40_T20m_TR2', '5GB1_FM40_T40m_TR1',
           '5GB1_FM40_T60m_TR1', '5GB1_FM40_T90m_TR2', '5GB1_FM40_T150m_TR1_remake', '5GB1_FM40_T180m_TR1']

df2_TPM = TPM_counts(df1_raw_FM40, "start_coord", "end_coord",columns)  #TPM counts
df2_TPM_log2 = log_2_transform(df2_TPM, "5GB1_FM40_T0m_TR2","5GB1_FM40_T180m_TR1") #TPM log 2 transformed 
df2_TPM_mean = mean_center(df2_TPM, "5GB1_FM40_T0m_TR2","5GB1_FM40_T180m_TR1") #TPM mean centered 

df3_pearson_r = congruency_table(df2_TPM, "5GB1_FM40_T0m_TR2" , "5GB1_FM40_T180m_TR1", step = df2_TPM.shape[0])
df3_euclidean_mean = euclidean_distance(df2_TPM_mean, "5GB1_FM40_T0m_TR2" , "5GB1_FM40_T180m_TR1")
df3_euclidean_log2 = euclidean_distance(df2_TPM_mean, "5GB1_FM40_T0m_TR2" , "5GB1_FM40_T180m_TR1" )

print("The shape of the TPM table is ", df2_TPM.shape)
print("The shape of the pearson_r matrix is ", df3_pearson_r.shape)



5G_counts.tsv was located in the directory
5G_counts.tsv was imported into dataframe
QC columns were removed from dataframe
All non FM40 data were removed from dataframe
All FM40 columns were sorted by timecourse sequence
Clean-up of raw data complete
The shape of the TPM table is  (4593, 16)
The shape of the pearson_r matrix is  (4593, 4593)


# Pearson R table gives N/A values for zeros

How to go about this, how many values does this apply to?

In [53]:
df3_pearson_r[0:1].isnull().sum().sum()

94

Looks like there are 94 N/A vlaues (0 gene expression). If there is zero gene expression that means a zero correlation, going to manually change these values to 0. 

In [71]:
df3_pearson_r_noNA = df3_pearson_r.fillna(value = 0)

In [67]:
#ceating a smaller dataframe with some null values
df_test = df3_pearson_r.iloc[4590:,4590:]

In [68]:
df_test

locus_tag,MBURv2_tRNA7,MBURv2_tRNA8,MBURv2_tRNA9
locus_tag,,,
MBURv2_tRNA7,1.000000,0.605269,NaN
MBURv2_tRNA8,0.605269,1.000000,NaN
MBURv2_tRNA9,NaN,NaN,NaN


In [69]:
df_test.fillna(value = 0)

locus_tag,MBURv2_tRNA7,MBURv2_tRNA8,MBURv2_tRNA9
locus_tag,,,
MBURv2_tRNA7,1.000000,0.605269,0
MBURv2_tRNA8,0.605269,1.000000,0
MBURv2_tRNA9,0.000000,0.000000,0


In [70]:
df_test.isnull()

locus_tag,MBURv2_tRNA7,MBURv2_tRNA8,MBURv2_tRNA9
locus_tag,,,
MBURv2_tRNA7,False,False,True
MBURv2_tRNA8,False,False,True
MBURv2_tRNA9,True,True,True


In [72]:
# Clustering the pearsons_R with N/A vlaues removed 

hdb_t1 = time.time()
hdb_pearson_r = hdbscan.HDBSCAN(metric = "precomputed", min_cluster_size=10).fit(df3_pearson_r_noNA)
hdb_pearson_r_labels = hdb.labels_
hdb_elapsed_time = time.time() - hdb_t1
print("time to cluster", hdb_elapsed_time)

time to cluster 5.757973909378052


In [73]:
np.unique(hdb_pearson_r_labels)

array([-1,  0,  1])

In [78]:
np.bincount(hdb_pearson_r_labels[hdb_pearson_r_labels!=-1])

array([26, 67])

# Clustering with HDBSCAN

In [9]:
import hdbscan
import time
from sklearn import metrics

In [28]:
# Clustering the mean centered euclidean distance of TPM counts 

hdb_t1 = time.time()
hdb_euclidean_mean = hdbscan.HDBSCAN(metric = "precomputed", min_cluster_size=10).fit(df3_euclidean_mean)
hdb_euclidean_mean_labels = hdb.labels_
hdb_elapsed_time = time.time() - hdb_t1
print("time to cluster", hdb_elapsed_time)

time to cluster 4.468245983123779


In [29]:
np.unique(hdb_euclidean_mean_labels)

array([-1,  0,  1])

In [79]:
np.bincount(hdb_euclidean_mean_labels[hdb_euclidean_mean_labels!=-1])

array([26, 67])

Lets try to convert dataframe to numpy array and see if the result is the same 

In [87]:
array3_euclidean_mean=np.array(df3_euclidean_mean)

In [88]:
hdb_t1 = time.time()
hdb_array_euclidean_mean = hdbscan.HDBSCAN(metric = "precomputed", min_cluster_size=10).fit(array3_euclidean_mean)
hdb_array_euclidean_mean_labels = hdb.labels_
hdb_elapsed_time = time.time() - hdb_t1
print("time to cluster", hdb_elapsed_time)

time to cluster 4.111801862716675


In [89]:
np.bincount(hdb_array_euclidean_mean_labels[hdb_array_euclidean_mean_labels!=-1])

array([26, 67])

looks like wether it is a numpy array or pandas dataframe, the result is the same. lets now try to get index of the clustered points. 

In [94]:
{i: np.where(hdb_array_euclidean_mean.labels_ == i)[0] for i in range(hdb_array_euclidean_mean.n_clusters)}

AttributeError: 'HDBSCAN' object has no attribute 'n_clusters'

In [37]:
# Clustering the log2 transformed euclidean distance of TPM counts 

hdb_t1 = time.time()
hdb_euclidean_log2 = hdbscan.HDBSCAN(metric = "precomputed", min_cluster_size=10).fit(df3_euclidean_log2)
hdb_euclidean_log2_labels = hdb.labels_
hdb_elapsed_time = time.time() - hdb_t1
print("time to cluster", hdb_elapsed_time)

time to cluster 3.162208080291748


In [36]:
np.unique(hdb_euclidean_log2_labels)

array([-1,  0,  1])

In [80]:
np.bincount(hdb_euclidean_log2_labels[hdb_euclidean_log2_labels!=-1])

array([26, 67])

### Following along to figure out the indices of my clusters. 

In [81]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
y = iris.target

estimator = KMeans(n_clusters=3)
estimator.fit(X)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=3, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [92]:
estimator.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2,
       0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2], dtype=int32)

In [93]:
{i: np.where(estimator.labels_ == i)[0] for i in range(estimator.n_clusters)}

{0: array([ 52,  77, 100, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112,
        115, 116, 117, 118, 120, 122, 124, 125, 128, 129, 130, 131, 132,
        134, 135, 136, 137, 139, 140, 141, 143, 144, 145, 147, 148]),
 1: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 2: array([ 50,  51,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
         64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 101, 106, 113, 114,
        119, 121, 123, 126, 127, 133, 138, 142, 146, 149])}